# Tournament Structure Functions

In [117]:
import pandas as pd
import numpy as np
import random
import math

def simulate_match(player1, player2, h2h_table):
    prob_p1 = h2h_table.loc[player1, player2]

    if prob_p1 == -1:
        return "invalid_bracket"
        
    return player1 if np.random.rand() < prob_p1 else player2

def simulate_bracket_with_placings(initial_bracket, h2h_table):
    current_round = initial_bracket
    placings = {competitor: None for competitor in h2h_table.index}  # Final placings
    eliminated_this_round = []

    round_number = 1
    while len(current_round) > 0:  # Simulate rounds until we get a winner
        current_place = len(current_round) * 2
        next_round = []
        for match in current_round:
            # Simulate the match
            winner = simulate_match(match[0], match[1], h2h_table)

            if winner == "invalid_bracket":
                return None
            
            loser = match[1] if winner == match[0] else match[0]
            next_round.append(winner)
            eliminated_this_round.append(loser)
        
        # Assign placings to eliminated players (if this was their final round)
        for player in eliminated_this_round:
            if placings[player] is None:
                placings[player] = current_place # Assign based on elimination order

        # Prepare for the next round
        current_round = [(next_round[i], next_round[i + 1]) for i in range(0, len(next_round), 2)] if len(next_round) > 1 else []
        eliminated_this_round.clear()
        round_number += 1

    # Assign the winner their placing (last player standing)
    if next_round:  # Check if there's still a player remaining
        winner = next_round[0]
        placings[winner] = 1

    return placings

# Function to generate a random bracket
def generate_random_bracket(h2h_table):
    competitors = list(h2h_table.index)
    random.shuffle(competitors)  # Shuffle competitors randomly

    # Limit competitors to the largest power of 2 less than or equal to the number of players
    max_power_of_2 = 2 ** int(math.log2(len(competitors)))
    limited_competitors = competitors[:max_power_of_2]

    # Pair competitors into matches
    bracket = [(limited_competitors[i], limited_competitors[i + 1]) for i in range(0, len(limited_competitors), 2)]

    return bracket

def simulate_multiple_brackets(h2h_table, num_simulations):
    results = []

    for _ in range(num_simulations):
        # Generate a random bracket
        random_bracket = generate_random_bracket(h2h_table)
        
        # Simulate the bracket and get placings
        placings = simulate_bracket_with_placings(random_bracket, h2h_table)
        
        # If the bracket is invalid (None), skip this iteration
        if placings is None:
            continue
        
        results.append(placings)

    # Convert results into a DataFrame
    results_df = pd.DataFrame(results)  # Do not fill missing values; keep NaN for players not in brackets
    return results_df

# Function to convert the raw H2H table to win probabilities
def convert_h2h_to_probs(h2h_raw, alpha=0, beta=0):
    h2h_probs = h2h_raw.copy()

    for row in h2h_probs.index:
        for col in h2h_probs.columns:
            raw_record = h2h_probs.loc[row, col]
            
            if raw_record == "0 - 0":
                # No matches played, set probability based on alpha and beta
                h2h_probs.loc[row, col] = 0.5 if alpha == 0 and beta == 0 else alpha / (alpha + beta)
            else:
                # Parse wins and losses
                wins, losses = map(int, raw_record.split(" - "))
                total = wins + losses
                
                if alpha == 0 and beta == 0:
                    # Frequentist approach: calculate win probability (wins/total)
                    h2h_probs.loc[row, col] = wins / total if total > 0 else 0.5
                else:
                    # Bayesian smoothing: (alpha + wins) / (alpha + beta + total)
                    h2h_probs.loc[row, col] = (alpha + wins) / (alpha + beta + total)
    
    return h2h_probs.astype(float)


# Real Data

In [118]:
import pandas as pd
h2h_record = pd.read_csv("./h2h_records/nightclub_s10_top30.csv")
h2h_record = h2h_record.rename(columns={"Unnamed: 0":"player"})

bracket_size = 20

h2h_sub = h2h_record.iloc[:bracket_size , :bracket_size+1]

h2h_sub = pd.DataFrame(h2h_sub).set_index("player")
h2h_sub

,aklo,epoodle,e-tie,gl!tch,daniel,freezus,tito jojo,tazio,k8a,just jason,fitzy,cannagar,danilo calamari,luu,abe,bonn,pgh fahey,da gobbler,tranimal,snoo
player,,,,,,,,,,,,,,,,,,,,
aklo,0 - 0,4 - 0,2 - 0,6 - 0,3 - 0,3 - 0,1 - 0,5 - 0,3 - 0,2 - 0,0 - 0,0 - 0,0 - 0,4 - 0,0 - 0,0 - 0,0 - 0,1 - 0,0 - 0,1 - 0
epoodle,0 - 4,0 - 0,5 - 1,1 - 1,0 - 0,2 - 2,1 - 1,2 - 0,2 - 0,0 - 0,1 - 0,0 - 0,0 - 0,1 - 0,0 - 0,2 - 0,0 - 0,0 - 0,1 - 0,0 - 0
e-tie,0 - 2,1 - 5,0 - 0,5 - 1,0 - 1,5 - 0,2 - 0,1 - 0,3 - 1,0 - 1,0 - 0,0 - 0,0 - 1,2 - 0,0 - 1,0 - 0,1 - 0,2 - 0,0 - 0,0 - 0
gl!tch,0 - 6,1 - 1,1 - 5,0 - 0,8 - 3,4 - 6,2 - 0,3 - 3,3 - 2,3 - 0,3 - 0,1 - 0,4 - 1,5 - 0,4 - 0,3 - 2,2 - 0,3 - 0,2 - 0,1 - 0
daniel,0 - 3,0 - 0,1 - 0,3 - 8,0 - 0,2 - 0,0 - 0,2 - 2,2 - 1,0 - 1,0 - 1,1 - 0,3 - 0,1 - 0,1 - 0,4 - 2,3 - 0,3 - 0,0 - 0,1 - 0
freezus,0 - 3,2 - 2,0 - 5,6 - 4,0 - 2,0 - 0,2 - 0,1 - 1,5 - 3,1 - 1,1 - 1,2 - 0,1 - 0,4 - 0,3 - 0,0 - 0,0 - 0,2 - 0,2 - 0,1 - 0
tito jojo,0 - 1,1 - 1,0 - 2,0 - 2,0 - 0,0 - 2,0 - 0,1 - 0,1 - 1,1 - 0,0 - 0,0 - 0,0 - 0,2 - 0,0 - 0,0 - 1,1 - 0,1 - 0,0 - 0,0 - 0
tazio,0 - 5,0 - 2,0 - 1,3 - 3,2 - 2,1 - 1,0 - 1,0 - 0,2 - 5,1 - 0,1 - 0,1 - 0,1 - 0,0 - 1,2 - 0,0 - 1,1 - 1,2 - 0,0 - 0,0 - 0
k8a,0 - 3,0 - 2,1 - 3,2 - 3,1 - 2,3 - 5,1 - 1,5 - 2,0 - 0,0 - 1,0 - 1,0 - 2,1 - 0,3 - 0,1 - 0,1 - 0,0 - 1,2 - 0,1 - 0,1 - 0


In [119]:
import pandas as pd
import numpy as np

# Create a mask where '0 - 0' records are replaced with NaN (except on the diagonal)
df_masked = h2h_record.copy()
df_masked = df_masked.set_index('player')

# Replace 0-0 values outside the diagonal with NaN
# Create a mask for '0 - 0' values on the non-diagonal elements
mask_zeros = (df_masked.values == '0 - 0')

# Set the diagonal elements of the mask to False
np.fill_diagonal(mask_zeros, False)

# Replace '0 - 0' values outside the diagonal with NaN
df_masked[:] = np.where(mask_zeros, np.nan, df_masked)
df_masked


,aklo,epoodle,e-tie,gl!tch,daniel,freezus,tito jojo,tazio,k8a,just jason,...,bambz,bigbuffalo,hatsune mitski,willy p,jango uu,kdog,wrap,kingnut,mudjam,moburu
player,,,,,,,,,,,,,,,,,,,,,
aklo,0 - 0,4 - 0,2 - 0,6 - 0,3 - 0,3 - 0,1 - 0,5 - 0,3 - 0,2 - 0,...,NaN,1 - 0,NaN,1 - 0,1 - 0,1 - 0,NaN,2 - 0,NaN,1 - 0
epoodle,0 - 4,0 - 0,5 - 1,1 - 1,NaN,2 - 2,1 - 1,2 - 0,2 - 0,NaN,...,1 - 0,1 - 0,NaN,NaN,1 - 0,1 - 0,NaN,NaN,NaN,NaN
e-tie,0 - 2,1 - 5,0 - 0,5 - 1,0 - 1,5 - 0,2 - 0,1 - 0,3 - 1,0 - 1,...,1 - 0,1 - 0,NaN,NaN,NaN,NaN,1 - 0,NaN,1 - 0,1 - 0
gl!tch,0 - 6,1 - 1,1 - 5,0 - 0,8 - 3,4 - 6,2 - 0,3 - 3,3 - 2,3 - 0,...,1 - 0,2 - 0,1 - 1,1 - 0,NaN,1 - 0,2 - 0,3 - 0,1 - 0,1 - 0
daniel,0 - 3,NaN,1 - 0,3 - 8,0 - 0,2 - 0,NaN,2 - 2,2 - 1,0 - 1,...,NaN,NaN,1 - 0,1 - 0,2 - 0,NaN,NaN,NaN,NaN,1 - 0
freezus,0 - 3,2 - 2,0 - 5,6 - 4,0 - 2,0 - 0,2 - 0,1 - 1,5 - 3,1 - 1,...,1 - 0,2 - 0,1 - 0,1 - 0,1 - 1,1 - 0,1 - 0,1 - 0,NaN,NaN
tito jojo,0 - 1,1 - 1,0 - 2,0 - 2,NaN,0 - 2,0 - 0,1 - 0,1 - 1,1 - 0,...,NaN,NaN,NaN,2 - 0,NaN,NaN,NaN,NaN,NaN,NaN
tazio,0 - 5,0 - 2,0 - 1,3 - 3,2 - 2,1 - 1,0 - 1,0 - 0,2 - 5,1 - 0,...,2 - 0,1 - 0,NaN,2 - 0,4 - 0,1 - 0,NaN,3 - 0,1 - 0,1 - 0
k8a,0 - 3,0 - 2,1 - 3,2 - 3,1 - 2,3 - 5,1 - 1,5 - 2,0 - 0,0 - 1,...,1 - 0,1 - 0,NaN,NaN,1 - 0,2 - 0,NaN,3 - 0,2 - 0,3 - 0


In [140]:
import networkx as nx

# Create a graph where each player is a node
G = nx.Graph()

# Add nodes (players)
players = df_masked.index
G.add_nodes_from(players)

# Add edges between players if they have a valid record (not '0 - 0')
for i in range(len(players)):
    for j in range(i+1, len(players)):
        if pd.notna(df_masked.iloc[i, j]) and pd.notna(df_masked.iloc[j, i]):
            G.add_edge(players[i], players[j])

# Use Bron–Kerbosch to find all maximal cliques
cliques = list(nx.algorithms.clique.find_cliques(G))

# Find the size of the largest cliques
max_size = max(len(clique) for clique in cliques)

# Get all cliques that are of the largest size and sort them based on the original order of players
largest_cliques = [sorted(clique, key=lambda player: players.get_loc(player)) for clique in cliques if len(clique) == max_size]

print(f"Largest subset size: {max_size}")
print(f"Largest subsets count: {len(largest_cliques)}")

# Now print the sorted cliques
for sorted_clique in largest_cliques:
    print(f"Largest subsets: {sorted_clique}")

Largest subset size: 10
Largest subsets count: 13
Largest subsets: ['aklo', 'gl!tch', 'freezus', 'tazio', 'k8a', 'just jason', 'luu', 'da gobbler', 'bigbuffalo', 'kdog']
Largest subsets: ['aklo', 'e-tie', 'gl!tch', 'freezus', 'tazio', 'k8a', 'just jason', 'luu', 'da gobbler', 'bigbuffalo']
Largest subsets: ['aklo', 'e-tie', 'gl!tch', 'freezus', 'tito jojo', 'tazio', 'k8a', 'just jason', 'luu', 'da gobbler']
Largest subsets: ['aklo', 'e-tie', 'gl!tch', 'daniel', 'freezus', 'tazio', 'k8a', 'just jason', 'luu', 'da gobbler']
Largest subsets: ['aklo', 'gl!tch', 'freezus', 'tazio', 'k8a', 'luu', 'da gobbler', 'bigbuffalo', 'kdog', 'kingnut']
Largest subsets: ['gl!tch', 'freezus', 'tazio', 'k8a', 'cannagar', 'danilo calamari', 'luu', 'da gobbler', 'kdog', 'kingnut']
Largest subsets: ['gl!tch', 'freezus', 'tazio', 'k8a', 'just jason', 'fitzy', 'danilo calamari', 'luu', 'da gobbler', 'kdog']
Largest subsets: ['gl!tch', 'freezus', 'tazio', 'k8a', 'just jason', 'danilo calamari', 'luu', 'da gobb

In [152]:
clique = largest_cliques[0]

h2h_largest_subset = df_masked.loc[clique, clique]
h2h_largest_subset

,aklo,gl!tch,freezus,tazio,k8a,just jason,luu,da gobbler,bigbuffalo,kdog
player,,,,,,,,,,
aklo,0 - 0,6 - 0,3 - 0,5 - 0,3 - 0,2 - 0,4 - 0,1 - 0,1 - 0,1 - 0
gl!tch,0 - 6,0 - 0,4 - 6,3 - 3,3 - 2,3 - 0,5 - 0,3 - 0,2 - 0,1 - 0
freezus,0 - 3,6 - 4,0 - 0,1 - 1,5 - 3,1 - 1,4 - 0,2 - 0,2 - 0,1 - 0
tazio,0 - 5,3 - 3,1 - 1,0 - 0,2 - 5,1 - 0,0 - 1,2 - 0,1 - 0,1 - 0
k8a,0 - 3,2 - 3,3 - 5,5 - 2,0 - 0,0 - 1,3 - 0,2 - 0,1 - 0,2 - 0
just jason,0 - 2,0 - 3,1 - 1,0 - 1,1 - 0,0 - 0,2 - 0,1 - 0,2 - 2,1 - 0
luu,0 - 4,0 - 5,0 - 4,1 - 0,0 - 3,0 - 2,0 - 0,4 - 1,0 - 2,1 - 1
da gobbler,0 - 1,0 - 3,0 - 2,0 - 2,0 - 2,0 - 1,1 - 4,0 - 0,0 - 1,1 - 2
bigbuffalo,0 - 1,0 - 2,0 - 2,0 - 1,0 - 1,2 - 2,2 - 0,1 - 0,0 - 0,2 - 0


In [153]:
# Convert raw H2H table to probabilities
h2h_probs = convert_h2h_to_probs(h2h_largest_subset, alpha = 0.5, beta = 0.5)
h2h_probs.round(2)

,aklo,gl!tch,freezus,tazio,k8a,just jason,luu,da gobbler,bigbuffalo,kdog
player,,,,,,,,,,
aklo,0.50,0.93,0.88,0.92,0.88,0.83,0.90,0.75,0.75,0.75
gl!tch,0.07,0.50,0.41,0.50,0.58,0.88,0.92,0.88,0.83,0.75
freezus,0.12,0.59,0.50,0.50,0.61,0.50,0.90,0.83,0.83,0.75
tazio,0.08,0.50,0.50,0.50,0.31,0.75,0.25,0.83,0.75,0.75
k8a,0.12,0.42,0.39,0.69,0.50,0.25,0.88,0.83,0.75,0.83
just jason,0.17,0.12,0.50,0.25,0.75,0.50,0.83,0.75,0.50,0.75
luu,0.10,0.08,0.10,0.75,0.12,0.17,0.50,0.75,0.17,0.50
da gobbler,0.25,0.12,0.17,0.17,0.17,0.25,0.25,0.50,0.25,0.38
bigbuffalo,0.25,0.17,0.17,0.25,0.25,0.50,0.83,0.75,0.50,0.83


In [154]:
# Example usage
num_simulations = 100000

# Simulate multiple brackets
simulation_results_df = simulate_multiple_brackets(h2h_probs, num_simulations)

# Calculate average placings
average_placings = simulation_results_df.mean().sort_values().round(2)

# Display results
print("\nAverage Placings:")
print(average_placings)


Average Placings:
aklo          2.56
gl!tch        4.56
freezus       4.66
k8a           5.05
tazio         5.33
just jason    5.44
bigbuffalo    5.87
kdog          6.62
luu           6.63
da gobbler    7.01
dtype: float64


In [151]:
simulation_results_df.shape

(100000, 10)